# Filter Debugging

I'm seeing the filter diverge after some time. This notebook tries to understand why this is happening.

In [ ]:
%matplotlib notebook

from experimental.beacon_sim import sim_log_pb2

import numpy as np
from spatialmath import SE2
from spatialmath.base.transforms2d import tr2xyt
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.style.use('ggplot')

np.set_printoptions(precision=6, linewidth=300)

import ipywidgets

import time

In [ ]:
# log_file_path = '/home/erick/scratch/beacon_sim.pb'
log_file_path = '/tmp/beacon_sim.pb'
log = sim_log_pb2.SimLog()
with open(log_file_path, 'rb') as file_in:
    log.ParseFromString(file_in.read())

def unpack_matrix(proto_msg):
    return np.array(proto_msg.data).reshape(proto_msg.num_rows, proto_msg.num_cols)
    
def unpack_estimate(proto_msg):
    out = {}
    out['mean'] = unpack_matrix(proto_msg.mean)
    out['cov'] = unpack_matrix(proto_msg.cov)
    out['beacon_ids'] = list(proto_msg.beacon_ids)
    return out

def unpack_observation(proto_msg):
    out = {}
    out['id'] = proto_msg.id
    out['range_m'] = proto_msg.range_m
    out['bearing_rad'] = proto_msg.bearing_rad
    return out

def unpack_observations(proto_msg):
    out = [unpack_observation(x) for x in proto_msg.observations]
    
    return out
    
def unpack_se2(proto_msg):
    real = proto_msg.rotation.complex.real
    imag = proto_msg.rotation.complex.imag
    trans = unpack_matrix(proto_msg.translation)
    
    mat = np.array([[real, -imag, trans[0, 0]], [imag, real, trans[1, 0]], [0.0, 0.0, 1.0]])
    
    return SE2(mat)
    
def parse_debug_message(proto_msg):
    out = {}
    out['time_of_validity'] = proto_msg.time_of_validity.ticks_since_epoch / 1e9
    out['prior'] = unpack_estimate(proto_msg.prior)
    out['old_from_new'] = unpack_se2(proto_msg.old_robot_from_new_robot)
    out['prediction'] = unpack_estimate(proto_msg.prediction)
    out['observations'] = unpack_observations(proto_msg.observations)
    out['posterior'] = unpack_estimate(proto_msg.posterior)
    out['local_from_true'] = unpack_se2(proto_msg.local_from_true_robot)
    out['measurement'] = unpack_matrix(proto_msg.measurement_vec)
    out['predicted_meas'] = unpack_matrix(proto_msg.prediction_vec)
    out['innovation'] = unpack_matrix(proto_msg.innovation_vec)
    out['observation_mat'] = unpack_matrix(proto_msg.observation_mat)
    return out
debug_msgs = [parse_debug_message(x) for x in log.debug_msgs]



In [ ]:
def get_ground_truth(debug_msgs):
    return SE2([x['local_from_true'] for x in debug_msgs])

def get_robot_estimate_mean(debug_msgs):
    return SE2([SE2(x=x['posterior']['mean'][0, 0],
                y=x['posterior']['mean'][1, 0],
                theta=x['posterior']['mean'][2]) for x in debug_msgs])

def get_covariance_matrix(debug_msgs):
    return np.stack([x['posterior']['cov'] for x in debug_msgs])

def get_observations(debug_msgs):
    out = {}
    
    for msg in debug_msgs:
        for obs in msg['observations']:
            if obs['id'] not in out:
                out[obs['id']] = {
                    'range_m': [],
                    'bearing_rad': []
                }
    
            out[obs['id']]['range_m'].append(obs['range_m'])
            out[obs['id']]['bearing_rad'].append(obs['bearing_rad'])
    
    return out

def get_beacon_estimates_mean(debug_msgs):
    out = {}
    for msg in debug_msgs:
        for i, beacon_idx in enumerate(msg['posterior']['beacon_ids']):
            if beacon_idx not in out:
                out[beacon_idx] = []
            out[beacon_idx].append(msg['posterior']['mean'][2 * i + 3:2 * i + 5, 0])
    for key, value in out.items():
        out[key] = np.vstack(value)
    return out

In [ ]:
def create_trajectory_collection(trajectory, x_color='#C00000', y_color='#00C000'):
    origin =  [x * np.array([[0.0, 0.0]]) for x in trajectory]
    px = [x * np.array([[0.25, 0.0]]) for x in trajectory]
    py = [x * np.array([[0.0, 0.25]]) for x in trajectory]
    line_collection = []
    colors = []
    for o, x, y in zip(origin, px, py):
        line_collection.append(np.vstack([o.T, x.T]))
        line_collection.append(np.vstack([o.T, y.T]))
        colors.append(x_color)
        colors.append(y_color)
    return mpl.collections.LineCollection(line_collection, colors=colors)


In [ ]:
gt = get_ground_truth(debug_msgs)
est = get_robot_estimate_mean(debug_msgs)
beacons = get_beacon_estimates_mean(debug_msgs)
fig, ax = plt.subplots()
gt_collection = ax.add_collection(create_trajectory_collection(gt))
est_collection = ax.add_collection(create_trajectory_collection(est, x_color='#E67E22', y_color='#3498DB'))


lines = []
for beacon, pts in beacons.items():
    lines.append(plt.plot(pts[:, 0], pts[:, 1], label=f'Beacon {beacon}')[0])
plt.xlim([-2, 10])
plt.ylim([-2, 10])
plt.title('Estimates over time vs gt')
plt.legend()
plt.tight_layout()


def upper_lim(idx):
    global gt_collection, est_collection, lines
    gt_collection.remove()
    est_collection.remove()
    for l in lines:
        l.remove()
    gt_collection = ax.add_collection(create_trajectory_collection(gt[:idx]))
    est_collection = ax.add_collection(create_trajectory_collection(est[:idx], x_color='#E67E22', y_color='#3498DB'))
    lines = []
    ax.set_prop_cycle(color=['red', 'green', 'blue', 'black'])
    for beacon, pts in beacons.items():
        lines.append(plt.plot(pts[:idx, 0], pts[:idx, 1], label = f'Beacon {beacon}')[0])
    plt.axis('equal')
    plt.legend()
    fig.canvas.draw()
    fig.canvas.flush_events()

# _ =ipywidgets.interact(upper_lim, idx=(1, len(beacons[0])))

for i in range(1, len(beacons[0])):
    upper_lim(i)
    time.sleep(0.05)
    fig.savefig(f'bug_{i:04}.png')


In [ ]:
plt.figure()
true_from_est = SE2([local_from_true.inv() * local_from_est for local_from_true, local_from_est in zip(gt, est)])

pos_error = true_from_est.t
abs_pos_error = np.sum(pos_error**2, axis = 1)

plt.plot(abs_pos_error)
plt.yscale('log')
plt.xlabel('Timestep')
plt.ylabel('Position Error')

In [ ]:
cov = get_covariance_matrix(debug_msgs)

plt.figure()
plt.subplot(211)
eig_vals = np.linalg.eigvalsh(cov)
condition_number = eig_vals[:, -1]  / eig_vals[:, 0]

plt.plot(condition_number)
plt.ylabel('Condition Number')
plt.xlabel('Timestep')
plt.yscale('log')

plt.subplot(212)
plt.plot(eig_vals[:, 0])

plt.tight_layout()

The condition numer is pretty high, but it's going down the entire time. I'll flag this for now, but I'll continue looking at the observations

In [ ]:
np.diag(cov[100,:,:])

It looks like the entry that corresponds to heading is the smallest

In [ ]:
labels = ['Robot X', 'Robot Y', 'Robot theta']
_ = [labels.extend([f'Beacon {beacon} {dim}' for dim in 'XY']) for beacon in beacons]

plt.figure()
for i, label in enumerate(labels):
    plt.plot(np.sqrt(cov[:, i, i]), label=label)
plt.legend()
plt.yscale('log')

In [ ]:
obs = get_observations(debug_msgs)

In [ ]:
plt.figure(figsize=(8, 6))
ranges = {beacon:np.array(values['range_m']) for beacon, values in obs.items()}
bearings = {beacon:np.array(values['bearing_rad']) for beacon, values in obs.items()}

plot_ax = plt.subplot(211)
for beacon, r in ranges.items():
    plt.plot(r, label=f'Beacon {beacon}')
plt.ylabel('Range')
plt.legend()

plt.subplot(212, sharex=plot_ax)
for beacon, b in bearings.items():
    plt.plot(b, label=f'Beacon {beacon}')
plt.ylabel('Bearing')
plt.xlabel('Timestep')
plt.legend()
plt.tight_layout()

So the last turn happens between timesteps 131 and 133. Timestep 133 is also when the covariance is smallest. The error starts growing really quickly after this point. Let's look at the heading errors.

In [ ]:
plt.figure()
theta_error = np.array(true_from_est.xyt())[:, -1]
plt.plot(theta_error)
plt.xlabel('Timestep')
plt.ylabel('Angle Error')

This seems to be a problem much later than position. I wonder if this somehow has something to do with wrap around... Playing around with the sim a bit more, it seems to be related to wrap around issues in the measurement update function. Moving in the -x direction of the local frame, makes it diverge almost immediately, but moving backwards along the +x direction seems to be okay.

In [ ]:
debug_msgs[160]